In [ ]:
!pip install langchain-huggingface
!pip install huggingface-hub
!pip install langchain-community
!pip install transformers
!pip install accelerate
!pip install bitsandbytes
!pip install langchain-core

In [2]:
import os
from google.colab import userdata
hf_token_id=userdata.get('HF_TOKEN')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = hf_token_id

In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
import torch

In [4]:
#loading the model in memory
#this is gated model. Go to Meta site and get access for Llama models
model_id="meta-llama/Llama-2-7b-chat-hf"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [6]:
quantization_config=BitsAndBytesConfig(
    load_in_4bit=True
)

In [7]:
model =  AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map="auto")

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [13]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

hf = HuggingFacePipeline(pipeline=pipe,pipeline_kwargs={"temperature": 0.2, "new_max_tokens": 100})

Device set to use cuda:0


In [14]:
hf

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7d7ea04b2410>, model_id='meta-llama/Llama-2-7b-chat-hf', pipeline_kwargs={'temperature': 0.2, 'new_max_tokens': 100})

In [15]:
hf.invoke("how are you?")

'how are you? I am writing to you from the future to tell you that the world has changed dramatically since you were alive. Unterscheidung between the two is not always easy, as both conditions can present with similar symptoms. The main difference between a panic attack and a heart attack is that a panic attack is not caused by a blockage in the coronary arteries, which supply blood to the heart. If you are experiencing symptoms of a panic attack or a heart attack, it is important to seek medical attention immediately. A heart attack occurs when the blood flow to the heart is blocked, causing damage to the heart muscle.\nThis can lead to severe chest pain or discomfort, shortness of breath, lightheadedness, and fatigue. Panic attacks, on the other hand, are episodes of intense fear or anxiety that can occur suddenly and without warning. They can cause a variety of physical symptoms, including a racing heart, sweating, trembling, and difficulty breathing. Both conditions can be treated

In [16]:
from langchain import PromptTemplate, LLMChain

In [17]:
question="how to perform EDA on IMDB moview reviews?"
# "who is a first president of INDIA?"
template="""Question: {queston}
give me a answer in detail manner and in step by step manner"""

prompt=PromptTemplate(template=template,input_variables=["question"])

In [18]:
llm_chain=LLMChain(llm=hf,prompt=prompt)

<ipython-input-18-01a75d341944>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain=LLMChain(llm=hf,prompt=prompt)


In [19]:
llm_chain.invoke(question)

{'queston': 'how to perform EDA on IMDB moview reviews?',
 'text': 'Question: how to perform EDA on IMDB moview reviews?\ngive me a answer in detail manner and in step by step manner.\n\nI\'m assuming you want to perform Exploratory Data Analysis (EDA) on IMDB movie reviews. Here\'s a step-by-step guide on how to perform EDA on IMDB movie reviews:\n\nStep 1: Data Collection\n\n* Go to the IMDB website and search for a movie you\'re interested in analyzing.\n* Click on the movie\'s title to view its page.\n* Scroll down to the "User Reviews" section and click on the "See all reviews" button.\n* You will be directed to a page showing all the reviews for that movie.\n* Click on the "Export" button to download the reviews in a CSV format.\n\nStep 2: Data Preparation\n\n* Once you have the reviews in a CSV file, you need to clean and preprocess the data.\n* Remove any irrelevant columns (e.g., movie title, release date, etc.) and keep only the columns that are relevant for EDA.\n* Convert t

In [20]:
from langchain.schema.runnable import RunnableSequence
from langchain_core.output_parsers import StrOutputParser

In [21]:
# parser = StrOutputParser()

In [29]:
chain = RunnableSequence(prompt,hf )

In [30]:
chain.invoke(question)

'Question: how to perform EDA on IMDB moview reviews?\ngive me a answer in detail manner and in step by step manner.\n\nI want to perform EDA on IMDB movie reviews to analyze the sentiment of the reviews. Here are the steps you can follow:\n\nStep 1: Data Collection\nThe first step is to collect the IMDB movie reviews data. You can use the IMDB API to fetch the reviews for a particular movie. The API provides a wide range of filters and options to help you customize your query. You can filter the reviews based on various criteria such as movie title, release date, rating, and more.\n\nStep 2: Preprocessing\nOnce you have collected the data, the next step is to preprocess it. This involves cleaning the data, handling missing values, and converting the data into a format suitable for analysis. Here are some steps you can follow:\n\na. Handle missing values: IMDB reviews may contain missing values, such as blank reviews or reviews with incomplete information. You can use a technique such 